<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/research/autogen_agent_hanoi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


FLAML offers an experimental feature of interactive LLM agents, which can be used to solve various tasks with human or automatic feedback, including tasks that require using tools via code.

`AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. By setting `human_input_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `AssistantAgent`. For example, when `human_input_mode` is set to "ALWAYS", the `UserProxyAgent` will always prompt the user for feedback. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `AssistantAgent` without doing any additional steps. When no user feedback is provided, the `UserProxyAgent` will execute the code written by `AssistantAgent` directly and return the execution results (success or failure and corresponding outputs) to `AssistantAgent`.


## Requirements

FLAML requires `Python>=3.8`. To run this notebook example, please install flaml with the [autogen] option:
```bash
pip install flaml[autogen]
```

In [1]:
# %pip install flaml[autogen]==2.0.0rc3

## Set your API Endpoint

The [`config_list_openai_aoai`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_openai_aoai) function tries to create a list of configurations using Azure OpenAI endpoints and OpenAI endpoints. It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure OpenAI API key + base.
The following code looks for key files in key_file_path. Change it into the relevant path. If you open this notebook in google colab, you can upload your files by click the file icon on the left panel and then choose "upload file" icon. Then change the key file path.




In [2]:
from flaml import oai

config_list = oai.config_list_openai_aoai(key_file_path="..")

## Construct Agents

We construct the assistant agent and the user proxy agent. We specify `human_input_mode` as "TERMINATE" in the user proxy agent, which will ask for feedback when it receives a "TERMINATE" signal from the assistant agent.

In [3]:
from flaml.autogen.agent import AssistantAgent, UserProxyAgent

# create an AssistantAgent instance named "assistant"
assistant = AssistantAgent(
    name="assistant",
    request_timeout=600,
    seed=41,
    config_list=config_list,
    model="gpt-4",  # make sure the endpoint you use supports the model
    temperature=0,
)
# create a UserProxyAgent instance named "user"
user = UserProxyAgent(
    name="user",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    work_dir='hanoi',
)

## Perform a task

We invoke the `receive()` method of the coding agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after the coding agent sends a "TERMINATE" signal in the end of the message. If you don't provide any feedback (by pressing Enter directly), the conversation will finish. Before the "TERMINATE" signal, the user proxy agent will try to execute the code suggested by the coding agent on behalf of the user.

In [5]:
assistant.reset()
assistant.receive(
    """Consider the following puzzle problem:

Problem description:
- There are three lists labeled A, B, and C.
- There are three numbers -- 0, 1, and 2 -- distributed among those three lists.
- You can move numbers from the end of one list to the end of another list.
Goal: The goal is to end up in the configuration where all numbers are in list C, in ascending order.
Rule #1: You can only move a number if it is at the end of its current list.
Rule #2: You can only move a number to the end of a list if it is larger than the other numbers in that list.

This is the starting configuration:
A = []
B = [0, 1]
C = [2]
This is the goal configuration:
A = []
B = []
C = [0, 1, 2]
Give me the sequence of moves to solve the puzzle from the starting configuration. Please try to use as few moves as possible, and make sure to follow the rules listed above.
Use heuristic search.
""",
    user
)

user (to assistant):

Consider the following puzzle problem:

Problem description:
- There are three lists labeled A, B, and C.
- There are three numbers -- 0, 1, and 2 -- distributed among those three lists.
- You can move numbers from the end of one list to the end of another list.
Goal: The goal is to end up in the configuration where all numbers are in list C, in ascending order.
Rule #1: You can only move a number if it is at the end of its current list.
Rule #2: You can only move a number to the end of a list if it is larger than the other numbers in that list.

This is the starting configuration:
A = []
B = [0, 1]
C = [2]
This is the goal configuration:
A = []
B = []
C = [0, 1, 2]
Give me the sequence of moves to solve the puzzle from the starting configuration. Please try to use as few moves as possible, and make sure to follow the rules listed above.
Use heuristic search.


--------------------------------------------------------------------------------
assistant (to user):

T